In [6]:
import cv2
import numpy as np
from object_detection import ObjectDetection
import math

In [7]:
cap = cv2.VideoCapture("traffic.mp4")

### Initializing object detection (Yolov4)

In [8]:
od = ObjectDetection()

Loading Object Detection
Running opencv dnn with YOLOv4


In [9]:
count = 0
center_points_cur_frame = []
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

### Detecting

In [10]:
while True:
    ret, frame = cap.read() 
    count += 1
    
    if not ret:
        break
    
    (class_ids, scores, boxes) = od.detect(frame)
    for box in boxes:
        (x, y, w, h) = box
        cx, cy = int((2 * x + w) / 2), int((2 * y + h) / 2)
        center_points_cur_frame.append((cx, cy))
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                
                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
        
    else:
        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()
        
        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue
            
            if not object_exists:
                tracking_objects.pop(object_id)
                
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1
    
    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255))
        
    
    cv2.imshow("Traffic", frame)
                    
    center_points_prev_frame = center_points_cur_frame.copy()
                    
    if cv2.waitKey(1) == ord('q'): 
        break
        
cap.release()
cv2.destroyAllWindows()